# DLRM - tensorflow Model 

In [34]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd
import sys
import os
from tqdm import tqdm
import logging
import seaborn as sns
from scipy import sparse
import matplotlib.pyplot as plt

In [57]:
from src.utils import LabelEncoding

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model

In [80]:
from tensorflow.keras.layers import Input, Dense, concatenate, Embedding, Flatten

In [37]:
device_name = tf.test.gpu_device_name()

In [38]:
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [7]:
data = pd.read_csv('./data/adult.csv')

In [8]:
data.shape

(48842, 15)

In [9]:
data.head(2)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K


In [10]:
all_columns = data.columns

In [30]:
num_cols = ['fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


In [31]:
cat_cols = list(set(all_columns)-set(num_cols))

In [32]:
target_cols = ['income']

In [33]:
print (num_cols)
print (cat_cols)
print (target_cols)

['fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
['gender', 'income', 'race', 'relationship', 'educational-num', 'native-country', 'marital-status', 'occupation', 'education', 'age', 'workclass']
['income']


## Model Architecture

##### Numerical Value (whole input)
- 512, 256, 16 with Relu
##### Categorical Value 
- embedding 16 for each input categori

##### Classification Layer 
- input : 16 * feature_num 
- 512, 256, 1 (sigmoid) 


In [35]:
class NumericalDense(Model):

    def __init__(self, units=[512, 256, 16], activation='relu', **kwargs):
        '''Initializes the class and sets up the internal variables'''
        super().__init__(**kwargs)
        
        self.layers = []
        
        for unit in units:
            layer = Dense(unit, activation=activation)
            self.layers.append(layer)
            
    def call(self, inputs):
        
        x = self.layers[0](inputs)
        for layer in self.layers[1:]:
            x = layer(x)
        
        return x

In [44]:
tf.one_hot([0,1,0,2], 5)

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

In [45]:
class CategoricalDense(Model):

    def __init__(self, feature_dic, embed_unit=16, activation='relu', **kwargs):
        '''Initializes the class and sets up the internal variables'''
        super().__init__(**kwargs)
        
        self.embed_layers = []
        self.feature_dic = feature_dic
        
        for _, item in feature_dic.items():
            layer = Embedding(item, embed_unit)
            self.embed_layers.append(layer)
            
    def call(self, inputs):
        
        input_oh = []
        output = []
        for idx, _input in enumerate(inputs):
            oh_ = tf.one_hot(_input, self.feature_dic[idx])
            input_oh.append(oh_)
        
        for layer, oh in zip(self.embed_layers, input_oh):
            output.append(layer(oh))

        return output

In [ ]:
class DLRM(Model):
    def __init__(self, units=[512, 256], activation='relu', **kwargs):
        '''Initializes the class and sets up the internal variables'''
        super(DLRM, self).__init__(**kwargs)
        
        self.layers = []
        
        for unit in units:
            layer = Dense(unit, activation=activation)
            self.layers.append(layer)
        
        output_layer = Dense(1, activation='softmax')
        self.layers.append(output_layer)
        
    def call(self, inputs)
        

In [47]:
num_data = data[num_cols]
cat_data = data[cat_cols]
target = data[target_cols]

In [102]:
target[target_cols] = (target[target_cols]=='>50K').astype(int)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [104]:
y = target[target_cols].values

In [105]:
X = [num_data.values]

In [106]:
cat_x = []
for col in cat_enc:
    cat_x.append(cat_enc[col].values)

In [107]:
X.append(cat_x)

In [50]:
num_data.shape[1]

4

In [51]:
cat_data.head(2)

,gender,income,race,relationship,educational-num,native-country,marital-status,occupation,education,age,workclass
0,Male,<=50K,Black,Own-child,7,United-States,Never-married,Machine-op-inspct,11th,25,Private
1,Male,<=50K,White,Husband,9,United-States,Married-civ-spouse,Farming-fishing,HS-grad,38,Private


In [52]:
for col in cat_data:
    print (col)

gender
income
race
relationship
educational-num
native-country
marital-status
occupation
education
age
workclass


In [58]:
lbenc = LabelEncoding()

In [59]:
lbenc.fit(cat_data, cat_data.columns)

In [61]:
cat_enc = lbenc.transform(cat_data)

In [62]:
cat_enc.head(2)

,gender,income,race,relationship,educational-num,native-country,marital-status,occupation,education,age,workclass
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,1,1,1,1,0


In [94]:

def Dlrm_model(num_inputs, 
               cat_inputs,
               feature_dic,
               num_units=[512,256, 16],
               embed_unit = 16,
               layer_units=[512, 256],
               activation='relu'):
    
    numerical_input = Input(shape=(num_inputs.shape[1],), name='Numerical Input')
    categorical_inputs = []
    embedded = []
    
    # One-hot encode the cat input data 
    for i, col in enumerate(cat_data):
        data_shape = cat_data[[col]].shape[1]
        cat_input = Input(shape=(data_shape,), name=f'Categorical Input_{i}')
        categorical_inputs.append(cat_input)
        #oh_ = tf.one_hot(cat_input, feature_dic[col])
        embed = Embedding(input_dim = feature_dic[col], output_dim=embed_unit)(cat_input)
        embed = Flatten()(embed)
        embedded.append(embed)

    # Dense networkd for numerical data 
    x = Dense(num_units[0], activation=activation)(numerical_input)
    for unit in num_units[1:]:
        x = Dense(unit, activation=activation)(x)
    
    # concat
    concat = concatenate([emb for emb in embedded], axis=-1)
    concat = concatenate([x, concat], axis=-1)
    
    # dense for concat layer 
    
    x = Dense(layer_units[0], activation=activation)(concat)
    for unit in layer_units[1:]:
        x = Dense(unit, activation=activation)(x)
    
    output = Dense(1, activation='softmax', name='Output')(x)
    
    model = Model(inputs=[numerical_input, categorical_inputs], outputs=output )
    
    return model
    
        

In [95]:
feature_dic = {}
for col in cat_enc:
    feature_dic[col] = cat_enc[col].nunique()

In [96]:
dlrm = Dlrm_model(num_data, cat_enc, feature_dic)

In [99]:
dlrm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [111]:
from sklearn.preprocessing import MinMaxScaler

In [115]:
scaler = MinMaxScaler()
num_data_scaled = scaler.fit_transform(num_data)

In [116]:
y = target[target_cols].values

In [117]:
X = [num_data_scaled]

In [118]:
cat_x = []
for col in cat_enc:
    cat_x.append(cat_enc[col].values)

In [119]:
X.append(cat_x)

In [120]:
dlrm.fit(X, y, epochs=10)

Epoch 1/10
1527/1527 [==============================] - 9s 5ms/step - loss: 2.4214e-08 - accuracy: 0.2393
Epoch 2/10
1527/1527 [==============================] - 8s 5ms/step - loss: 1.1979e-08 - accuracy: 0.2393
Epoch 3/10
1527/1527 [==============================] - 8s 5ms/step - loss: 6.7966e-09 - accuracy: 0.2393
Epoch 4/10
1527/1527 [==============================] - 8s 5ms/step - loss: 4.2547e-09 - accuracy: 0.2393
Epoch 5/10
1527/1527 [==============================] - 8s 5ms/step - loss: 2.8673e-09 - accuracy: 0.2393
Epoch 6/10
1527/1527 [==============================] - 8s 5ms/step - loss: 2.0312e-09 - accuracy: 0.2393
Epoch 7/10
1527/1527 [==============================] - 8s 5ms/step - loss: 1.4501e-09 - accuracy: 0.2393
Epoch 8/10
1527/1527 [==============================] - 8s 5ms/step - loss: 1.0536e-09 - accuracy: 0.2393
Epoch 9/10
1527/1527 [==============================] - 8s 5ms/step - loss: 7.6777e-10 - accuracy: 0.2393
Epoch 10/10
1527/1527 [=======================

In [27]:
class WideAndDeepModel(Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        '''initializes the instance attributes'''
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1, name='output')
        self.aux_output = Dense(1, name='aux_output')
        
    def call(self, inputs):
        '''defines the network architecture'''
        input_A, input_B = inputs
        input_A_tensor = Input(input_A)
        input_B_tensor = Input(input_B)
        
        hidden1 = self.hidden1(input_B_tensor)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_A_tensor, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        
        self.input_A_tensor = input_A_tensor
        self.input_B_tensor = input_B_tensor
        self.main_output = main_output
        self.aux_output = aux_output
        
        return main_output, aux_output
    
    def summary(self):
        self.model = Model(inputs = [self.input_A_tensor, self.input_B_tensor ], 
                           outputs = [self.main_output, self.aux_output])
        self.model.summary()
        

In [28]:
model = WideAndDeepModel()

In [29]:
model.summary()

AttributeError: 'WideAndDeepModel' object has no attribute 'input_A_tensor'

In [3]:

input_layer = Input(shape=(8,))
x = Dense(64, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
## Please uncomment all lines in this cell and replace those marked with `# YOUR CODE HERE`.
## You can select all lines in this code cell with Ctrl+A (Windows/Linux) or Cmd+A (Mac), then press Ctrl+/ (Windows/Linux) or Cmd+/ (Mac) to uncomment.



class SimpleQuadratic(Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the class and sets up the internal variables'''
        super(SimpleQuadratic, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
        # YOUR CODE HERE
    
    def build(self, input_shape):
        
        '''Create the state of the layer (weights)'''
        a_init = tf.random_normal_initializer()
        self.a = tf.Variable(name="kernel_a",
            initial_value=a_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        
        b_init = tf.random_normal_initializer()
        self.b = tf.Variable(name="kernel_b",
            initial_value=b_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        
        
        c_init = tf.zeros_initializer()
        self.c = tf.Variable(name="bias",
            initial_value=c_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        super().build(input_shape)
        # a and b should be initialized with random normal, c (or the bias) with zeros.
        # remember to set these as trainable.
        # YOUR CODE HERE
   
    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        # YOUR CODE HERE
        value = tf.matmul(tf.math.square(inputs), self.a) + tf.matmul(inputs, self.a) + self.c
        value = self.activation(value)
        return value